# Libraries

In [21]:
import json
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Reading and arranging data

In [3]:
file_path = '/content/train_dataset.json'
with open(file_path, 'r') as json_file:
    dataset = json.load(json_file)

main_dict = dataset["conversation"]


#labels contains emotions
#data contains the text

initial_labels = []
og_data = []

for sub_dict in main_dict.values():
    for inner_dict in sub_dict:
        emotion = inner_dict.get('emotion')
        text = inner_dict.get('text')
        initial_labels.append(emotion)
        og_data.append(text)



# Create a mapping from emotions to numbers
emotion_mapping = {
    "neutral": 0,
    "joy": 1,
    "surprise": 2,
    "disgust": 3,
    "sadness": 4,
    "anger": 5,
    "fear": 6
}

labels = [emotion_mapping[emotion] for emotion in initial_labels]


del file_path,json_file,dataset,emotion,text,main_dict,sub_dict,inner_dict,initial_labels,emotion_mapping

# Data preprocessing

In [4]:
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def clean_text(text):
    text = text.lower()

    text = re.sub(r"[^\w\s]", "", text)

    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word.lower() not in stop_words]

    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]

    cleaned_text = " ".join(lemmatized_text)

    return cleaned_text

In [6]:
data = [clean_text(phrase) for phrase in og_data]

# Check data integrity
***don't run this***

In [7]:
for original, cleaned in zip(og_data[14:20], data[14:20]):
    print(f"Original: {original}\nCleaned: {cleaned}\n---")

Original: Yeah , but what if it is not ? What if there is a reason why we can not have a baby ?
Cleaned: yeah reason baby
---
Original: Oh , Chandler , look . You and Monica are meant to have children . I am sure it is gonna be just fine .
Cleaned: oh chandler look monica meant child sure gon na fine
---
Original: oh , oh , yeah , ok , thanks .
Cleaned: oh oh yeah ok thanks
---
Original: I can not believe I did not even think of that .
Cleaned: believe even think
---
Original: I guess I was just so worried about having to ... come here and do ... that ...
Cleaned: guess worried come
---
Original: What , you can do it in the parking lot of a Taco Bell , but you can not do it at a doctor office ?
Cleaned: parking lot taco bell doctor office
---


In [8]:
print(len(data),len(labels))

10246 10246


In [9]:
print(set(labels))

{0, 1, 2, 3, 4, 5, 6}


# ML stuff

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [15]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# SVC

In [16]:
svm_classifier = SVC(kernel='linear')
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear']}
grid_search = GridSearchCV(svm_classifier, param_grid, refit=True, verbose=3, cv=3)
grid_search.fit(X_train_tfidf, y_train)
#svm_classifier.fit(X_train_tfidf, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.465 total time=   2.2s
[CV 2/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.463 total time=   2.2s
[CV 3/3] END .....C=0.1, gamma=1, kernel=linear;, score=0.461 total time=   2.5s
[CV 1/3] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.465 total time=   2.1s
[CV 2/3] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.463 total time=   2.1s
[CV 3/3] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.461 total time=   2.1s
[CV 1/3] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.465 total time=   2.2s
[CV 2/3] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.463 total time=   2.6s
[CV 3/3] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.461 total time=   2.1s
[CV 1/3] END .C=0.1, gamma=0.001, kernel=linear;, score=0.465 total time=   2.1s
[CV 2/3] END .C=0.1, gamma=0.001, kernel=linear;, score=0.463 total time=   2.1s
[CV 3/3] END .C=0.1, gamma=0.001, kernel=linear;

GridSearchCV(cv=3, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear']},
             verbose=3)

In [18]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [19]:
y_pred = grid_search.predict(X_test_tfidf)

In [20]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.92      0.64       907
           1       0.57      0.27      0.36       341
           2       0.66      0.29      0.40       265
           3       0.57      0.06      0.11        63
           4       0.36      0.08      0.13       168
           5       0.43      0.09      0.15       257
           6       0.00      0.00      0.00        49

    accuracy                           0.51      2050
   macro avg       0.44      0.24      0.26      2050
weighted avg       0.50      0.51      0.43      2050



# Random Forest

In [22]:
rf_classifier = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]}
grid_search = GridSearchCV(rf_classifier, param_grid, refit=True, verbose=3, cv=3)
grid_search.fit(X_train_tfidf, y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV 1/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.471 total time=   6.1s
[CV 2/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.469 total time=   6.0s
[CV 3/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.462 total time=   5.4s
[CV 1/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.471 total time=  11.4s
[CV 2/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.469 total time=  11.5s
[CV 3/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.467 total time=  11.5s
[CV 1/3] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.469 total t

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             verbose=3)

In [23]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


In [24]:
y_pred = grid_search.predict(X_test_tfidf)

In [25]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.89      0.64       907
           1       0.53      0.27      0.36       341
           2       0.54      0.35      0.43       265
           3       0.33      0.02      0.03        63
           4       0.36      0.08      0.13       168
           5       0.37      0.09      0.14       257
           6       0.00      0.00      0.00        49

    accuracy                           0.50      2050
   macro avg       0.38      0.24      0.25      2050
weighted avg       0.47      0.50      0.43      2050



# NN LSTM

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [27]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(data)

In [28]:
vocab_size = len(tokenizer.word_index) + 1

In [29]:
sequences = tokenizer.texts_to_sequences(data)

In [30]:
max_sequence_length = 100

In [31]:
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [32]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [34]:
embedding_dim = 50  # Choose an appropriate embedding dimension
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(7, activation='softmax')  # Adjust the number of units based on your number of classes
])

In [35]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
257/257 [==============================] - 16s 48ms/step - loss: 1.6168 - accuracy: 0.4340 - val_loss: 1.5894 - val_accuracy: 0.4424
Epoch 2/5
257/257 [==============================] - 4s 14ms/step - loss: 1.6032 - accuracy: 0.4340 - val_loss: 1.5916 - val_accuracy: 0.4424
Epoch 3/5
257/257 [==============================] - 3s 12ms/step - loss: 1.6030 - accuracy: 0.4340 - val_loss: 1.5906 - val_accuracy: 0.4424
Epoch 4/5
257/257 [==============================] - 2s 9ms/step - loss: 1.6031 - accuracy: 0.4340 - val_loss: 1.5899 - val_accuracy: 0.4424
Epoch 5/5
257/257 [==============================] - 2s 8ms/step - loss: 1.6024 - accuracy: 0.4340 - val_loss: 1.5918 - val_accuracy: 0.4424


In [39]:
# Evaluate the model
y_pred_probabilities = model.predict(X_test)
y_pred = tf.argmax(y_pred_probabilities, axis=1)

65/65 [==============================] - 0s 5ms/step


# TASK 2 ⚡



In [ ]:
#pip install datasets

In [73]:
import json
from transformers import BertTokenizer, BertForSequenceClassification, BartForConditionalGeneration, BartTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
import torch
import torch.optim as optim
import pandas as pd
import nltk
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_metric
import numpy as np

In [74]:
def read_json_and_create_dataframes(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    utterance_rows = []
    pair_rows = []
    full_convo_texts = {}
    full_convo_texts_tagged = {}

    for conversation_id, conversation in data['conversation'].items():
        full_convo_texts[conversation_id] = ""
        full_convo_texts_tagged[conversation_id] = ""

        for utterance in conversation:
            full_convo_texts[conversation_id] += utterance["text"] + " "
            full_convo_texts_tagged[conversation_id] += f"{utterance['utterance_ID']}_{utterance['text']} "

            utterance_row = {
                "conversation_id": conversation_id,
                "utterance_id": utterance["utterance_ID"],
                "speaker": utterance["speaker"],
                "emotion": utterance["emotion"],
                "text": utterance["text"]
            }
            utterance_rows.append(utterance_row)

        if conversation_id in data['emotion-cause_pairs']:
            for pair in data['emotion-cause_pairs'][conversation_id]:
                emotion_id, cause_text = pair
                utterance_id, emotion = emotion_id.split("_")
                pair_row = {
                    "conversation_id": conversation_id,
                    "emotion_utterance_id": utterance_id,
                    "cause_text": cause_text
                }
                pair_rows.append(pair_row)

    df_utterances = pd.DataFrame(utterance_rows)
    df_pairs = pd.DataFrame(pair_rows)

    df_full_convos = pd.DataFrame(list(full_convo_texts.items()), columns=['conversation_id', 'full_conversation'])
    df_full_convos_tagged = pd.DataFrame(list(full_convo_texts_tagged.items()), columns=['conversation_id', 'full_conversation_tagged'])

    df_utterances['utterance_id'] = df_utterances['utterance_id'].astype(str)
    df_pairs['emotion_utterance_id'] = df_pairs['emotion_utterance_id'].astype(str)

    df_merged = pd.merge(df_utterances, df_pairs, how='left', left_on=['conversation_id', 'utterance_id'], right_on=['conversation_id', 'emotion_utterance_id'], suffixes=('', '_pair'))
    df_final = pd.merge(df_merged, df_full_convos, on='conversation_id', how='left')
    df_final = pd.merge(df_final, df_full_convos_tagged, on='conversation_id', how='left')

    df_final['cause_text'] = df_final['cause_text'].fillna('')

    df_final = df_final.drop(columns=['emotion_utterance_id'])

    return df_final

In [75]:
file_path = '/content/train_dataset.json'
file_path_test = '/content/test_dataset.json'

df_conversations = read_json_and_create_dataframes(file_path)
df_test_conversations = read_json_and_create_dataframes(file_path_test)

In [76]:
class EmotionCauseDatasetQA_BERT(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        full_conversation = row['full_conversation']
        text = f"{row['utterance_id']}_{row['text']}"
        cause_text = row['cause_text'] if pd.notnull(row['cause_text']) else ""

        inputs = self.tokenizer.encode_plus(
            f"{full_conversation} [SEP] {text} [SEP] ",
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = torch.tensor(1 if cause_text else 0, dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [77]:
class EmotionCauseDatasetBART(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        full_conversation = row['full_conversation']
        target_phrase = f"{row['utterance_id']}_{row['text']}"
        cause_text = f"<EMOTION>{row['utterance_id']}_{row['emotion']}<CAUSE>{row['cause_text']}" if pd.notnull(row['cause_text']) else '<NO_CAUSE>'

        prompt = f"Given the following conversation where each phrase is labeled with its ID: {full_conversation}. Considering the specific statement: '{target_phrase}', with its emotion, identify and label the part of the conversation that likely caused this emotion."

        inputs = self.tokenizer(prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        targets = self.tokenizer(cause_text, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        labels = targets["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [78]:
def train_bart_model(model, dataloader, device, epochs=2):
    optimizer = AdamW(model.parameters(), lr=5e-5)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, batch in enumerate(dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
            loss = outputs.loss
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(dataloader)}, Loss: {loss.item()}")

        avg_train_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}, Average Loss: {avg_train_loss}")

def safe_decode_predictions(token_ids, tokenizer):
    token_ids = token_ids[token_ids != -100]
    return tokenizer.decode(token_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [79]:
def evaluate_predictions_on_id(predictions, true_labels):
    def extract_id(text):
        id_end_index = text.find('_')
        if id_end_index != -1:
            return text[:id_end_index].strip()
        return ''

    correct_count = 0
    total_count = len(predictions)

    for pred, true in zip(predictions, true_labels):
        pred_id = extract_id(pred)
        true_id = extract_id(true)

        if pred_id == true_id:
            correct_count += 1

    accuracy = correct_count / total_count if total_count > 0 else 0
    print(f"Accuracy based on ID comparison: {accuracy:.4f} ({correct_count}/{total_count})")

In [80]:
def evaluate_bart_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    for batch_idx, batch in enumerate(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        outputs = model.generate(input_ids, attention_mask=attention_mask)
        pred_texts = [safe_decode_predictions(g, tokenizer) for g in outputs]
        true_texts = [safe_decode_predictions(t.cpu().numpy(), tokenizer) for t in batch['labels']]

        predictions.extend(pred_texts)
        true_labels.extend(true_texts)

        if (batch_idx + 1) % 10 == 0:
            print(f"Evaluating Batch {batch_idx+1}/{len(dataloader)}")
    exact_matches = sum([1 for pred, true in zip(predictions, true_labels) if pred == true])
    exact_match_rate = exact_matches / len(predictions)
    print(f"Exact Match Rate: {exact_match_rate}")
    evaluate_predictions_on_id(predictions, true_labels)

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer_bart = BartTokenizer.from_pretrained('facebook/bart-base')
model_bart.to(device)

train_dataset_bart = EmotionCauseDatasetBART(df_conversations, tokenizer_bart)
train_dataloader_bart = DataLoader(train_dataset_bart, batch_size=4, shuffle=True)

test_dataset_bart = EmotionCauseDatasetBART(df_test_conversations, tokenizer_bart)
test_dataloader_bart = DataLoader(test_dataset_bart, batch_size=4)

# Train and evaluate
train_bart_model(model_bart, train_dataloader_bart, device, epochs=2)
evaluate_bart_model(model_bart, test_dataloader_bart, device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Batch 10/3056, Loss: 1.401691198348999
Epoch 1, Batch 20/3056, Loss: 0.6228248476982117
Epoch 1, Batch 30/3056, Loss: 0.5766474008560181
Epoch 1, Batch 40/3056, Loss: 0.6674779057502747
Epoch 1, Batch 50/3056, Loss: 0.6233906149864197
Epoch 1, Batch 60/3056, Loss: 0.35687586665153503
Epoch 1, Batch 70/3056, Loss: 0.409392774105072
Epoch 1, Batch 80/3056, Loss: 0.08458410948514938
Epoch 1, Batch 90/3056, Loss: 0.5095883011817932
Epoch 1, Batch 100/3056, Loss: 0.2792995572090149
Epoch 1, Batch 110/3056, Loss: 0.23057983815670013
Epoch 1, Batch 120/3056, Loss: 0.33956968784332275
Epoch 1, Batch 130/3056, Loss: 0.2817486524581909
Epoch 1, Batch 140/3056, Loss: 0.3259899318218231
Epoch 1, Batch 150/3056, Loss: 0.6772845983505249
Epoch 1, Batch 160/3056, Loss: 0.1391163468360901
Epoch 1, Batch 170/3056, Loss: 0.17441587150096893
Epoch 1, Batch 180/3056, Loss: 0.42194464802742004
Epoch 1, Batch 190/3056, Loss: 0.26048049330711365
Epoch 1, Batch 200/3056, Loss: 0.485975980758667
Epoch

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluating Batch 10/1000
Evaluating Batch 20/1000
Evaluating Batch 30/1000
Evaluating Batch 40/1000
Evaluating Batch 50/1000
Evaluating Batch 60/1000
Evaluating Batch 70/1000
Evaluating Batch 80/1000
Evaluating Batch 90/1000
Evaluating Batch 100/1000
Evaluating Batch 110/1000
Evaluating Batch 120/1000
Evaluating Batch 130/1000
Evaluating Batch 140/1000
Evaluating Batch 150/1000
Evaluating Batch 160/1000
Evaluating Batch 170/1000
Evaluating Batch 180/1000
Evaluating Batch 190/1000
Evaluating Batch 200/1000
Evaluating Batch 210/1000
Evaluating Batch 220/1000
Evaluating Batch 230/1000
Evaluating Batch 240/1000
Evaluating Batch 250/1000
Evaluating Batch 260/1000
Evaluating Batch 270/1000
Evaluating Batch 280/1000
Evaluating Batch 290/1000
Evaluating Batch 300/1000
Evaluating Batch 310/1000
Evaluating Batch 320/1000
Evaluating Batch 330/1000
Evaluating Batch 340/1000
Evaluating Batch 350/1000
Evaluating Batch 360/1000
Evaluating Batch 370/1000
Evaluating Batch 380/1000
Evaluating Batch 390/